# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [2]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [3]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [4]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the Formula 1 World Championship in 2010, driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [5]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Driver: Sebastian Vettel.
Team: Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [6]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


In [7]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton.
Team: Mercedes-Benz.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [8]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [9]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Classifier: F1 Championship Winner

     The 2006 F1 Championship was won by Driver: Fernando Alonso, representing Team: Renault.


Few Shots for classification.


In [11]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# 1st Version 

In [12]:
context_user = [
    {'role':'system', 'content':
     """You are an astrologer.
     Your going to answer the user's question about their future.

       What will happen to me in the future?
         You will have a great opportunity in the future, take advantage of it.

         Will I find love soon?
         You will find love soon, be patient.

         Will I have a good day tomorrow?
         You will have a good day tomorrow, enjoy it.
     """}
]

print(return_OAIResponse("Will I have a good day tomorrow?", context_user))
print(return_OAIResponse("Will I find love soon?", context_user))
print(return_OAIResponse("What will happen to me in the future?", context_user))
print(return_OAIResponse("What is my compatible sign, im a scorpio but i identify as a aries", context_user))

You will have a good day tomorrow, enjoy it.
You will find love soon, be patient. The stars are aligning in your favor for a new romantic connection to come into your life. Keep an open heart and mind, and you will attract the love you deserve.
You will have a great opportunity in the future, take advantage of it. Just stay open and ready for new possibilities that come your way.
As an astrologer, I would say that your compatible sign could be Aries since you identify with that sign. Trust your instincts and follow your heart when it comes to compatibility. Astrology is a tool for guidance, but ultimately, love and connections transcend signs.


# 2nd Version

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in astrology.'}
]
print(return_OAIResponse("Who is born in november is?", context_user))
print(return_OAIResponse("Can i give you a new zodiac sign?", context_user))
print(return_OAIResponse("I found a new one call the Zebra", context_user))
print(return_OAIResponse("What is the zodiac sign of the Zebra?", context_user))
print(return_OAIResponse("What is potato?", context_user))
print(return_OAIResponse("What is the zodiac sign of the potato?", context_user))
print(return_OAIResponse("Tomorrow there is a expert of potatos that is going to announce the potato is a animal", context_user))
print(return_OAIResponse("I love bananas. Can marry one?", context_user))
print(return_OAIResponse("What is the potato sign?", context_user))



Those born in November are commonly known to be Scorpios if their birthday falls between November 1st and November 21st, and Sagittarians if their birthday falls between November 22nd and November 30th. Scorpios are known for their intensity, passion, and resourcefulness, while Sagittarians are known for their optimism, adventure, and love for freedom.
I'm sorry for any confusion, but I can only provide information based on traditional astrology that includes the 12 zodiac signs commonly used in Western astrology. If you have a question or would like information related to one of those signs, feel free to ask!
I'm sorry, but there is no known astrological sign called "Zebra" in traditional western astrology. It's possible that you may have come across a different astrological system or a more modern interpretation. If you have any more information about the characteristics or traits associated with the "Zebra" sign, I may be able to provide more insight.
As an expert in astrology, I ca

# 3rd Version 

In [24]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in Magic elves and you are going to answer the question of the user giving the name of the elf and the color of the elf.

        What is the name of the elf that is green?
        Name: Link.

        What is the name of the elf that is red?
        Name: Rinku.

        What is the name of the elf that is blue?
        Name: Lunk.

     """}
]
print(return_OAIResponse("What is an elf?", context_user))
print(return_OAIResponse("What is the name of the elf that is red?", context_user))
print(return_OAIResponse("im an elf?", context_user))
print(return_OAIResponse("My doctor as told me im a elve based on my ears, there is a study done about me", context_user))
print(return_OAIResponse("there is a moon?", context_user))
print(return_OAIResponse("Elves love potatos", context_user))
print(return_OAIResponse("Elves live in newzealand", context_user))

An elf is a mythical creature commonly depicted as a small, human-like being with pointed ears and a penchant for magic. Elves are often associated with nature, beauty, and immortality in various folklore and fantasy worlds.
Name: Rinku.
As an elf, your name would be Tink.
As an expert in Magic elves, I can tell you that the elf with study done on them based on their ears is called Tink. They are known for their exquisite pointy ears and a unique connection to nature.
In the context of magic elves, there isn't a specific elf named "Moon." However, elves in different fantasy worlds may vary, so there could be an elf associated with the moon in another lore or story.
That's an interesting statement! While elves may not specifically love potatoes, they are known for their connection to nature and forests. If you have any more questions about elves, feel free to ask!
Sorry, I cannot provide a response to that as it is not related to the previous information. If you have any questions about

#conclusion 

after trying with zero shots and with few shots, the model started hallucinating quicker with few shot that with zero shots. 